In [ ]:
print("hello")

hello


In [3]:
aws_profile_name = "dev"
aws_region = "us-east-1"
import boto3
from langchain_aws import ChatBedrockConverse

boto3.setup_default_session(profile_name=aws_profile_name)
BEDROCK_CLIENT = boto3.client("bedrock-runtime", aws_region)
llm = ChatBedrockConverse(
    client=BEDROCK_CLIENT,
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    max_tokens=1000,
    temperature=0.2,
    top_p=0.9,
)

In [17]:
from langchain_mcp_adapters.client import MultiServerMCPClient,SSEConnection

async with MultiServerMCPClient(
    {
        "math": {
            "transport": "sse",
            "timeout": 1.0,
            "sse_read_timeout": 1.0,
            "url": "http://localhost:8000/sse",
            "headers": None,
            "session_kwargs": None
        }
    }
) as client:
    print(client.get_tools())

[StructuredTool(name='add', description='Add two numbers', args_schema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7ff00ad3bba0>), StructuredTool(name='multiply', description='Multiply two numbers', args_schema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiplyArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7ff00ad3b7e0>)]


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient,SSEConnection

client = MultiServerMCPClient(
    {
        "math": {
            "transport": "sse",
            "timeout": 1.0,
            "sse_read_timeout": 1.0,
            "url": "http://localhost:8000/sse",
            "headers": None,
            "session_kwargs": None
        }
    }
)

TypeError: object MultiServerMCPClient can't be used in 'await' expression

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode


def call_model(state: MessagesState):
    print("Calling Model")
    print(state)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                    Use tools when needed instead of asking permission.
                    Don't reference the tool use in your response content.
                """.strip(),
            ),
            *state["messages"],
        ]
    )
    chain = prompt | llm.bind_tools(client.get_tools())
    response = chain.invoke({})
    return {"messages": [response]}


tool_node = ToolNode(client.get_tools())

In [6]:
import json
from typing import cast

from langchain_core.messages import AIMessage
from langgraph.graph import END, MessagesState


def should_call_tool_or_end(state: MessagesState):
    last_message = cast(AIMessage, state["messages"][-1])
    if last_message.tool_calls:
        print(f"Calling tool(s): {json.dumps(last_message.tool_calls, indent=2)}")
        return tool_node.name
    return END

In [11]:
graph = StateGraph(MessagesState)

# definitions
graph.add_node(call_model)
graph.add_node(tool_node)

# navigation
graph.set_entry_point(call_model.__name__)
graph.add_conditional_edges(
    call_model.__name__,
    should_call_tool_or_end,
)
graph.add_edge(tool_node.name, call_model.__name__)

agent = graph.compile()
agent.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
       +------------+         
       | call_model |         
       +------------+         
         .         .          
       ..           ..        
      .               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


In [13]:
await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

Calling Model
{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='1022870b-8b10-40ed-a641-cdfd2b57bd91')]}


{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='1022870b-8b10-40ed-a641-cdfd2b57bd91'),
  AIMessage(content="To calculate (3 + 5) x 12:\n\nFirst, let's solve what's inside the parentheses:\n3 + 5 = 8\n\nNow we can multiply that result by 12:\n8 x 12 = 96\n\nTherefore, the final answer is 96.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b8d83680-8a11-4a16-aedf-e485aa4e5ecc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 20 Apr 2025 23:10:32 GMT', 'content-type': 'application/json', 'content-length': '366', 'connection': 'keep-alive', 'x-amzn-requestid': 'b8d83680-8a11-4a16-aedf-e485aa4e5ecc'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1449]}, 'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-5cadecf7-8b27-4925-9629-cdfdfbf9b907-0', usage_metadata={'input_tokens': 43, 'output_tokens': 71, 'total_tokens': 114, 'input_token_details': {'cache_cr